## Setup

In [1]:
# Load packages
using DifferentialEquations
using Plots, StatsPlots
using CSV, DataFrames
using Turing
using LaTeXStrings
using XLSX
using Optim
using StatsBase
using Random
using KernelDensity
using ProgressMeter
using Distributions
using Measures
using GLM

# Setup plots standard
Plots.default(fontfamily = ("computer modern"))

# Set seed 
Random.seed!(42)

TaskLocalRNG()

In [2]:
# Add rhs file
include("model_rhs.jl")
    
# Add parameter file
include("model_default_param.jl")

# Add treatment rhs
include("model_rhs_treatment.jl")

# Add function for calculating VAF
include("model_calc_VAF.jl")

# Add model for Turing inference - individual
include("model_infer_dynamics.jl")

# Add model for Turing inference - hierarchical
include("model_infer_dynamics_Hierarchical_Gibbs_DALIAH.jl")

# Add model function for Turing optimisation
include("model_infer_dynamics_post_Hierarchical.jl")

model_infer_dynamics_post_Hierarchical

In [3]:
# Setup initial conditions for master curve
x00 = 1.0*10^5
x10 = 2.5*10^6
x20 = 6.4*10^11
y00 = 1
y10 = 0
y20 = 0
a0 = 8.1*10^2
s0 = 1

# Collect in one vector
u0 = [x00,x10,x20,y00,y10,y20,a0,s0]

# Setup and solve ODEproblem
tspan = (0.0,90*365)
prob = ODEProblem(model_rhs, u0, tspan, p)
sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

# Save as named tuple
master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))

(t = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0  …  32841.0, 32842.0, 32843.0, 32844.0, 32845.0, 32846.0, 32847.0, 32848.0, 32849.0, 32850.0], x0 = [100000.0, 99999.28014604042, 99998.58099317514, 99997.90054517318, 99997.2361247567, 99996.58543544661, 99995.94650693233, 99995.31764842379, 99994.69740877306, 99994.0845423813  …  1.8609987268004509, 1.8600391708260122, 1.859080109644587, 1.8581215430323301, 1.8571634706299531, 1.8562058922481908, 1.855248807523938, 1.8542922162899567, 1.85333611836694, 1.8523805134158782], x1 = [2.5e6, 2.499892552031063e6, 2.4997848422058923e6, 2.4996769420471084e6, 2.4995689453848847e6, 2.4994609323745994e6, 2.4993529714592863e6, 2.4992451210456532e6, 2.4991374309370755e6, 2.499029943558893e6  …  58.85223884536521, 58.82189390263089, 58.791564607190864, 58.761250952072494, 58.73095292584728, 58.700670522809894, 58.67040373102239, 58.640152545142634, 58.609916959557594, 58.57969696360518], x2 = [6.4e11, 6.406571135075917e11, 6.412179399621925e11,

## Data

In [4]:
# Setup dataframes
df_D = DataFrame

# Load data
path = "M:/data_cancer/DALIAH/Trine_DALIAH_5y/DALIAH_5y - Export for Julia.csv"
df_D = CSV.read(path, df_D)

# Change data types
df_D[!,:days] = convert.(Float64,df_D[:,:days])
df_D[!,:RUX] = convert.(Float64,df_D[:,:RUX])


df_D

Row,patientID,days,IFN,RUX,JAK,TRC,WBC,studyVisit
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2,-22.0,0.0,0.0,NaN,NaN,6.3,-1.0
2,2,0.0,4.28571,0.0,0.49,300.0,7.9,0.0
3,2,14.0,4.28571,0.0,NaN,321.0,6.4,0.5
4,2,28.0,4.28571,0.0,NaN,211.0,6.1,1.0
5,2,56.0,4.28571,0.0,NaN,232.0,4.9,2.0
6,2,119.0,4.28571,0.0,0.46,225.0,5.4,4.0
7,2,182.0,4.28571,0.0,NaN,181.0,4.8,6.0
8,2,238.0,7.14286,0.0,0.32,161.0,4.0,8.0
9,2,294.0,7.14286,0.0,NaN,131.0,4.0,10.0


In [5]:
# Convert to starting at time for first data point - no matter if JAK or treatment
for i in unique(df_D.patientID)
    # Set pID
    pID = i
    
    # Extract minimum number of days - where IFN>0 or JAK>=0
    df_p = df_D[df_D.patientID .== pID,:]

    minDays = minimum(df_p.days[df_p.IFN .> 0  .|| df_p.JAK .>= 0, :])
    
    # Subtract minimum number of days
    df_D.days[df_D.patientID .== pID, :] = df_D.days[df_D.patientID .== pID, :] .- minDays
end
df_D

Row,patientID,days,IFN,RUX,JAK,TRC,WBC,studyVisit
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2,-22.0,0.0,0.0,NaN,NaN,6.3,-1.0
2,2,0.0,4.28571,0.0,0.49,300.0,7.9,0.0
3,2,14.0,4.28571,0.0,NaN,321.0,6.4,0.5
4,2,28.0,4.28571,0.0,NaN,211.0,6.1,1.0
5,2,56.0,4.28571,0.0,NaN,232.0,4.9,2.0
6,2,119.0,4.28571,0.0,0.46,225.0,5.4,4.0
7,2,182.0,4.28571,0.0,NaN,181.0,4.8,6.0
8,2,238.0,7.14286,0.0,0.32,161.0,4.0,8.0
9,2,294.0,7.14286,0.0,NaN,131.0,4.0,10.0


In [6]:
# Define new patients
# new_patients = [22,112,132,163,179,200]
# new_patients = [11,20,44,84,85,103,108,111,114,137,139,145,147,149]
new_patients = [6,16,25,27,44,45,82,102,108,122,126,160,179,200]

14-element Vector{Int64}:
   6
  16
  25
  27
  44
  45
  82
 102
 108
 122
 126
 160
 179
 200

## Calculate for all patients and all data points

In [7]:
# Unique patients
unique_patients = new_patients

# Number of patients
P = length(unique_patients)

# Plot for all patients in loop
@showprogress dt=1 desc="Calculating results for all patients and all data points..." for i=1:P 
    # Extract relevant data
    pID = unique_patients[i]
    df_p = df_D[df_D.patientID .== pID, :]
    
    # Extract latest data point
    maxDays = maximum(df_p.days)

    # Extract VAF
    pVAF = df_p.JAK
    
    # Use only data at which VAF is not NaN
    pVAF = pVAF[.!isnan.(pVAF)]
    
    # Length of pVAF
    J = length(pVAF)

    # Choose treatment effect
    effect = "py0dy1"

    # Choose time for calculating VAF
    pDays = 0:1:7*365
    pDays2 = df_p.days[.!isnan.(df_p.JAK)]

    # Chooase to estimate initJAK as well
    estInitJAK = true
    
    # Sample in loop over number of data points
    for j=1:J
        # Extract VAF
        pVAF = df_p.JAK
        
        # Use only data at which VAF is not NaN
        pVAF = pVAF[.!isnan.(pVAF)]

        # Use choose how many data points to use
        nData = j
        pVAF = pVAF[1:nData]

        # Instantiate model
        model = model_infer_dynamics(pVAF,df_p,effect,p,master_curve,estInitJAK)

        # Set seed 
        Random.seed!(42)

        # Sample using Turing
        MCMC = sample(model, HMC(0.05, 10), 10500)

        # Convert to data frame
        df_MCMC = DataFrame(MCMC)
        
        path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Pre Hierarchical/MCMC_py0dy1_p$(pID)_$(j).csv"
        CSV.write(path, df_MCMC)
    end
end

Sampling: 100%|█████████████████████████████████████████| Time: 0:07:29
Sampling: 100%|█████████████████████████████████████████| Time: 0:07:32
Sampling: 100%|█████████████████████████████████████████| Time: 0:07:34
Sampling: 100%|█████████████████████████████████████████| Time: 0:08:03
Sampling: 100%|█████████████████████████████████████████| Time: 0:09:45
Sampling: 100%|█████████████████████████████████████████| Time: 0:08:40
Sampling: 100%|█████████████████████████████████████████| Time: 0:08:18
Sampling: 100%|█████████████████████████████████████████| Time: 0:07:51
Sampling: 100%|█████████████████████████████████████████| Time: 0:07:01
Sampling: 100%|█████████████████████████████████████████| Time: 0:07:01
Sampling: 100%|█████████████████████████████████████████| Time: 0:06:31
Sampling: 100%|█████████████████████████████████████████| Time: 0:05:57
Sampling: 100%|█████████████████████████████████████████| Time: 0:06:05
Sampling: 100%|█████████████████████████████████████████| Time: 

Sampling: 100%|█████████████████████████████████████████| Time: 0:05:50
Sampling: 100%|█████████████████████████████████████████| Time: 0:05:44
Sampling: 100%|█████████████████████████████████████████| Time: 0:05:38
Sampling: 100%|█████████████████████████████████████████| Time: 0:05:31
Sampling: 100%|█████████████████████████████████████████| Time: 0:05:21
Sampling: 100%|█████████████████████████████████████████| Time: 0:05:13
Calculating results for all patients and all data points... 100% Time: 11:40:37


## Plot for all patients and all data points

In [8]:
# Unique patients
unique_patients = new_patients

# Number of patients
P = length(unique_patients)

# Maximum number of observations
maxObs = 9

# Storage
RMSE_mat = NaN*ones(P,maxObs)
meanCIwidth_mat = NaN*ones(P,maxObs)

# Plot for all patients in loop
@showprogress dt=1 desc="Plotting results for all patients and all data points..." for i=1:P 
    # Extract relevant data
    pID = unique_patients[i]
    df_p = df_D[df_D.patientID .== pID, :]
    
    # Extract latest data point
    maxDays = maximum(df_p.days)

    # Extract VAF
    pVAF = df_p.JAK
    
    # Use only data at which VAF is not NaN
    pVAF = pVAF[.!isnan.(pVAF)]
    
    # Length of pVAF
    J = length(pVAF)

    # Choose treatment effect
    effect = "py0dy1"

    # Choose time for calculating VAF
    pDays = 0:1:75*30.4
    pDays2 = df_p.days[.!isnan.(df_p.JAK)]
    pVAF2 = df_p.JAK[.!isnan.(df_p.JAK)]
    pDays_plot = pDays[1:30:end]

    # Chooase to estimate initJAK as well
    estInitJAK = true
    
    # Plot in loop for all data points
    for j=1:J
        # Extract VAF
        pVAF = df_p.JAK
        
        # Use only data at which VAF is not NaN
        pVAF = pVAF[.!isnan.(pVAF)]

        # Use choose how many data points to use
        nData = j
        pVAF = pVAF[1:nData]

        # Load results
        df_MCMC = DataFrame
        path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Pre Hierarchical/MCMC_py0dy1_p$(pID)_$(j).csv"
        df_MCMC = CSV.read(path, df_MCMC)
        
        # Remove burn-in
        deleteat!(df_MCMC,collect(1:500))

        # Storage
        VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC[:,"iteration"]))
        VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC[:,"iteration"]))
        p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC[:,"iteration"]), 8)

        # Calculate VAF in loop
        @showprogress dt=1 desc="Calculating VAF for MCMC chain..." for k in eachindex(df_MCMC.rho1)
            # Set rho values
            rho = [df_MCMC.rho1[k], df_MCMC.rho2[k]]

            # Set initial JAK
            initJAK = df_MCMC.initJAK[k]

            # Calculate VAF using function
            VAF_mat[:,k], sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
            VAF_mat_points[:,k], sol_points = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)
            p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
                       a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
            p_mat[:,k,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
        end
        VAF_mat_plot = VAF_mat[1:30:end,:]

        # Calculate RMSE based on mean at the points
        RMSE = rmsd(median(VAF_mat_points, dims=2),df_p.JAK[.!isnan.(df_p.JAK)])
        
        # Save result
        RMSE_mat[i,j] = RMSE
        
        # Calculate mean width of 95% CI
        mean_width = mean(abs.(quantile.(eachrow(VAF_mat), 0.975)*100-quantile.(eachrow(VAF_mat), 0.025)*100))
        
        # Save result
        meanCIwidth_mat[i,j] = mean_width

        # Calculate mean parameter values
        median_param_1 = median(df_MCMC[:,"rho1"])
        median_param_2 = median(df_MCMC[:,"rho2"])
        
        # Plot results
        figVAF = plot(df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
            linewidth = 3, label = "", colour= :blue1)
        plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
          linewidth = 3, label = "", colour= :blue1)
        title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho="
               *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
        xlabel!(L"t"*"/months")
        ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
        xlims!(0, 75)
        ylims!(0, 20)

        p2 = twinx()
        plot!(p2, pDays_plot/30.4, quantile.(eachrow(VAF_mat_plot), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat_plot), 0.975)*100, fillalpha = 0.35, 
             colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
#         plot!(p2, pDays[pDays.<=maxDays]/30.4, mean(VAF_mat[pDays.<=maxDays,:],dims=2)*100, linewidth = 3, 
#                  label = L"JAK2"*" VAF - Mean", colour = :darkgoldenrod1)
#         plot!(p2, pDays[pDays.>maxDays]/30.4, mean(VAF_mat[pDays.>maxDays,:],dims=2)*100, linewidth = 3, 
#               label = L"JAK2"*" VAF - Mean prediction", colour= :darkgoldenrod1, linestyle = :dash)
        plot!(p2, pDays_plot[pDays_plot.<=maxDays]/30.4, median(VAF_mat_plot[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 3, 
              label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod1)
        plot!(p2, pDays_plot[pDays_plot.>maxDays]/30.4, median(VAF_mat_plot[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 3, 
              label = L"JAK2"*" VAF - Median prediction", colour = :darkgoldenrod1, linestyle = :dash)
        scatter!(p2, pDays2[1:nData]./30.4, pVAF2[1:nData]*100, label=L"\textrm{Data}", markercolor=:red, markersize=4, markerstrokewidth=0.25)
        scatter!(p2, pDays2[nData+1:end]./30.4, pVAF2[nData+1:end]*100, label=L"\textrm{Data}", markercolor=:magenta3, markershape=:diamond, markersize=4, markerstrokewidth=0.25)
        ylabel!(p2, L"JAK2"*" VAF/%")
        xlims!(p2, 0, 75)
        ylims!(p2, 0, 100)
        plot!(titlefont=18,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        
        # Save figure
        figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Individual/MCMC - Pre Hierarchical/"
        savefig(figVAF,figpath*"Individual_MCMC_$(effect)_pre_Hierarchical_$(pID)_$(j).png")
        savefig(figVAF,figpath*"Individual_MCMC_$(effect)_pre_Hierarchical_$(pID)_$(j).pdf")
        savefig(figVAF,figpath*"Individual_MCMC_$(effect)_pre_Hierarchical_$(pID)_$(j).svg")
        
        # Plot density
        l = @layout [a a; a a; a a; a a;]
        plot_rho1_chain = plot(df_MCMC.iteration,df_MCMC.rho1,linewidth = 0.5, colour = 1, label = "")
        title!("\nChain for "*L"\rho_{s_{y_0}}")
        xlabel!("Iteration")
        ylabel!(L"\rho/1")
        plot_rho1_density = plot(density(df_MCMC.rho1,legend=false),linewidth = 0.5, colour = 1, label = "")
        xlims!(0, 1)
        title!("\nDensity for "*L"\rho")
        xlabel!(L"\rho/1")
        ylabel!("pdf/1")
        plot_rho2_chain = plot(df_MCMC.iteration,df_MCMC.rho2,linewidth = 0.5, colour = 1, label = "")
        title!("Chain for "*L"\delta")
        xlabel!("Iteration")
        ylabel!(L"\rho_{p_{y_1}}/\textrm{day}^{-1}")
        plot_rho2_density = plot(density(df_MCMC.rho2,legend=false),linewidth = 0.5, colour = 1, label = "")
        xlims!(0, 1)
        title!("Density for "*L"\delta")
        xlabel!(L"\rho_{p_{y_1}}/\textrm{day}^{-1}")
        ylabel!("pdf/1")
        plot_initJAK_chain = plot(df_MCMC.iteration,df_MCMC.initJAK*100,linewidth = 0.5, colour = 1, label = "")
        title!("Chain for Initial VAF")
        xlabel!("Iteration")
        ylabel!("Initial VAF/%")
        plot_initJAK_density = plot(density(df_MCMC.initJAK*100,legend=false),linewidth = 0.5, colour = 1, label = "")
        xlims!(0, 100)
        title!("Density for "*"Initial VAF")
        xlabel!("Initial VAF/%")
        ylabel!("pdf/1")
        plot_sigma_chain = plot(df_MCMC.iteration,df_MCMC.sigma*100, linewidth = 0.5, colour = 1, label = "")
        title!("Chain for "*L"\sigma")
        xlabel!("Iteration")
        ylabel!(L"\sigma/\%")
        plot_sigma_density = plot(density((df_MCMC.sigma*100),legend=false),linewidth = 0.5, colour = 1, label = "")
        xlims!(0, 60)
        title!("Density for "*L"\sigma")
        xlabel!(L"\sigma/\%")
        ylabel!("pdf/1")
        figMCMC = plot(plot_rho1_chain,plot_rho1_density,plot_rho2_chain,plot_rho2_density,plot_initJAK_chain,plot_initJAK_density,
            plot_sigma_chain,plot_sigma_density,layout = l,size = (700, 1000))
        
        # Save figure
        figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Individual/MCMC - Pre Hierarchical/"
        savefig(figMCMC,figpath*"Chain_Individual_MCMC_$(effect)_pre_Hierarchical_$(pID)_$(j).png")
        savefig(figMCMC,figpath*"Chain_Individual_MCMC_$(effect)_pre_Hierarchical_$(pID)_$(j).pdf")
        savefig(figMCMC,figpath*"Chain_Individual_MCMC_$(effect)_pre_Hierarchical_$(pID)_$(j).svg")
    end
end

Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:24
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:33
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:20
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:22
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:28
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:21
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:21
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:20
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:13
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:12
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:08
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:03
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:01:02
Calculating VAF for MCMC chain... 100%|████████████

Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:00:42
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:00:41
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:00:40
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:00:40
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:00:37
Calculating VAF for MCMC chain... 100%|██████████████████| Time: 0:00:36
Plotting results for all patients and all data points... 100% Time: 1:37:05


In [9]:
## Storage
mean_RMSE_vec = NaN*ones(maxObs)
mean_meanCIwidth_vec = NaN*ones(maxObs)

# Calculate means in loop
for i=1:maxObs
    temp = RMSE_mat[:,i]
    mean_RMSE_vec[i] = mean(temp[.!(isnan.(temp))])
    temp = meanCIwidth_mat[:,i]
    mean_meanCIwidth_vec[i] = mean(temp[.!(isnan.(temp))])
end

# Convert to dataframe
df_RMSE = DataFrame(RMSE_mat, :auto)
df_CI = DataFrame(meanCIwidth_mat, :auto)

# Save results
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/RMSE_test_train.csv"
CSV.write(path, df_RMSE)
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/CI_test_train.csv"
CSV.write(path, df_CI)

"C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/CI_test_train.csv"

In [10]:
RMSE_mat

14×9 Matrix{Float64}:
 0.0289888  0.019093    0.0175531    …    0.00499748   NaN
 0.0710668  0.0829734   0.0637446         0.0134863      0.013315
 0.0571071  0.0266955   0.019553        NaN            NaN
 0.0335393  0.0323048   0.0285445         0.0206518      0.0213776
 0.0439434  0.0345067   0.0493884       NaN            NaN
 0.053719   0.058015    0.0693719    …  NaN            NaN
 0.0778378  0.0788986   0.0766703         0.0150085    NaN
 0.215167   0.199434    0.184693          0.0371853      0.0302845
 0.462542   0.415974    0.372506          0.0533578      0.0493272
 0.0520122  0.0595652   0.0519894       NaN            NaN
 0.246054   0.14672     0.161052     …    0.0586088      0.0550678
 0.0745665  0.0589781   0.0595803         0.0326155      0.0304088
 0.173739   0.159455    0.076973        NaN            NaN
 0.0481826  0.00139112  0.000817242       0.000487254    0.000402284

In [ ]:
meanCIwidth_mat